In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt

# Data Loading and Analysis

In [2]:
yogurt_df = pd.read_csv("../datasets/Yogurt100N.csv")

In [3]:
yogurt_df.describe()

,Pan I.D.,Expend $,Income,HH Size,IPT,Quantity,Brand 1,Brand 2,Brand 3,Brand 4,Feature 1,Feature 2,Feature 3,Feature 4,Price 1,Price 2,Price 3,Price 4,PanelistFirstObs
count,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000,2430.000000
mean,48.607819,36.476062,8.720988,2.802058,4.533745,2.585597,0.341975,0.401235,0.029218,0.227572,0.055556,0.039506,0.037449,0.037449,0.106248,0.081532,0.053622,0.079507,0.041152
std,27.858116,31.673291,3.800654,1.173291,14.930892,1.869089,0.474469,0.490249,0.168452,0.419351,0.229109,0.194836,0.189897,0.189897,0.020587,0.011047,0.008054,0.007714,0.198683
min,1.000000,-0.100000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.012000,0.000000,0.025000,0.004000,0.000000
25%,23.000000,12.092500,6.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103000,0.081000,0.050000,0.079000,0.000000
50%,47.000000,27.485001,9.000000,3.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.108000,0.086000,0.054000,0.079000,0.000000
75%,73.000000,51.712500,12.000000,4.000000,5.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.115000,0.086000,0.061000,0.086000,0.000000
max,100.000000,286.720000,14.000000,6.000000,358.000000,14.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.193000,0.111000,0.086000,0.104000,1.000000


In [4]:
yogurt_df.head()

,Pan I.D.,Expend $,Income,HH Size,IPT,Quantity,Brand 1,Brand 2,Brand 3,Brand 4,Feature 1,Feature 2,Feature 3,Feature 4,Price 1,Price 2,Price 3,Price 4,PanelistFirstObs
0,1,40.900002,9,2,5,2,0,0,0,1,0,0,0,0,0.108,0.081,0.061,0.079,1
1,1,16.809999,9,2,5,2,0,1,0,0,0,0,0,0,0.108,0.098,0.064,0.075,0
2,1,4.060000,9,2,1,2,0,1,0,0,0,0,0,0,0.108,0.098,0.061,0.086,0
3,1,34.459999,9,2,4,2,0,1,0,0,0,0,0,0,0.108,0.098,0.061,0.086,0
4,1,8.390000,9,2,7,2,0,1,0,0,0,0,0,0,0.125,0.098,0.049,0.079,0


# Baseline model

In [5]:
"""
fun callable
The objective function to be minimized.

fun(x, *args) -> float

where x is a 1-D array with shape (n,) and args is a tuple of the fixed parameters needed to completely specify the function.
"""
def optimization_func(x):
    a1, a2, a3, bf, bp = x
    v1 = np.exp(a1 + bf*yogurt_df['Feature 1'] + bp*yogurt_df['Price 1'])
    v2 = np.exp(a2 + bf*yogurt_df['Feature 2'] + bp*yogurt_df['Price 2'])
    v3 = np.exp(a3 + bf*yogurt_df['Feature 3'] + bp*yogurt_df['Price 3'])
    v4 = np.exp(0 + bf*yogurt_df['Feature 4'] + bp*yogurt_df['Price 4'])
    denom = v1 + v2 + v3 + v4
    loss = (v1*yogurt_df['Brand 1'] + v2*yogurt_df['Brand  2'] + v3*yogurt_df['Brand 3'] + v4*yogurt_df['Brand 4'])/denom
    log_loss = np.log(loss)
    total_ll = np.sum(log_loss)
    
    return -total_ll

In [104]:
a1, a2, a3, bf, bp = 1, 1, 1, 1, 1
x_init = np.array([a1, a2, a3, bf, bp])
output = opt.minimize(optimization_func, x_init, options={'gtol': 0.001})

In [58]:
output.x

array([  1.38833517,   0.64329837,  -3.0792272 ,   0.48751425,
       -37.09431806])

In [105]:
output

      fun: 2658.556697507448
 hess_inv: array([[ 7.31431810e-03,  2.19894144e-03, -2.60613505e-03,
        -2.02728035e-03, -1.48218164e-01],
       [ 2.19894144e-03,  3.03292937e-03,  1.60753512e-03,
        -1.29048511e-04, -1.13511244e-02],
       [-2.60613505e-03,  1.60753512e-03,  1.98742582e-02,
         1.56499141e-03,  1.54833107e-01],
       [-2.02728035e-03, -1.29048511e-04,  1.56499141e-03,
         1.26688626e-02,  6.20651477e-02],
       [-1.48218164e-01, -1.13511244e-02,  1.54833107e-01,
         6.20651477e-02,  5.24561718e+00]])
      jac: array([ 0.00000000e+00,  0.00000000e+00, -1.22070312e-04, -1.22070312e-04,
       -3.05175781e-05])
  message: 'Optimization terminated successfully.'
     nfev: 114
      nit: 16
     njev: 19
   status: 0
  success: True
        x: array([  1.38775775,   0.64350504,  -3.08612187,   0.48741097,
       -37.05809013])

The parameter estimates from the optimizer are\
a1 = 1.38775208\
a2 = 0.64350485\
a3 = -3.08611349\
bf = 0.48741416\
bp = -37.05787954

In [21]:
max_log_likelihood = -output.fun

In [22]:
max_log_likelihood

-2658.556697505227

# Nested Logit model

### Case 1: 3 brands in one bucket and the last in another

In [183]:
def nested_optimization_func(x):
    a1, a2, a3, bf, bp, theta = x
    rho = np.exp(theta)/(np.exp(theta)+1)
    v1 = np.exp((a1 + bf*yogurt_df['Feature 1'] + bp*yogurt_df['Price 1']) / rho)
    v2 = np.exp((a2 + bf*yogurt_df['Feature 2'] + bp*yogurt_df['Price 2']) / rho)
    v3 = np.exp((a3 + bf*yogurt_df['Feature 3'] + bp*yogurt_df['Price 3']) / rho)
    v4 = np.exp(bf*yogurt_df['Feature 4'] + bp*yogurt_df['Price 4'])
    denom = v1 + v2 + v3
    
    global P1, P2, P3, P4
    
    P4 = v4 / (np.power(denom, rho) + v4)
    P1 = (1 - P4) * v1 / denom
    P2 = (1 - P4) * v2 / denom
    P3 = (1 - P4) * v3 / denom

    loss = (P1 * yogurt_df['Brand 1'] + P2 * yogurt_df['Brand  2'] + P3 * yogurt_df['Brand 3'] + P4 * yogurt_df['Brand 4'])
    log_loss = np.log(loss)
    total_ll = np.sum(log_loss)
    
    return -total_ll

In [184]:
a1, a2, a3, bf, bp, theta = 1, 1, 1, 1, 1, 1
x_init = np.array([a1, a2, a3, bf, bp, theta])
output = opt.minimize(nested_optimization_func, x_init, options={'gtol': 0.001})

In [185]:
output.x

array([  1.38167812,   0.83941322,  -1.65858047,   0.37447103,
       -26.58196683,   0.59016408])

In [167]:
output

      fun: 2653.764600012444
 hess_inv: array([[ 5.68284288e-03,  7.23662873e-04, -1.21312961e-02,
        -7.56063197e-04, -1.89546689e-01,  1.18517553e-02],
       [ 7.23662873e-04,  5.33585131e-03,  2.62917653e-02,
        -1.67351175e-03,  2.14535402e-01, -2.56335978e-02],
       [-1.21312961e-02,  2.62917653e-02,  2.10449297e-01,
        -1.19541285e-02,  1.89847482e+00, -2.09180668e-01],
       [-7.56063197e-04, -1.67351175e-03, -1.19541285e-02,
         7.54488746e-03, -7.07308160e-02,  1.30472472e-02],
       [-1.89546689e-01,  2.14535402e-01,  1.89847482e+00,
        -7.07308160e-02,  1.93384556e+01, -1.87806123e+00],
       [ 1.18517553e-02, -2.56335978e-02, -2.09180668e-01,
         1.30472472e-02, -1.87806123e+00,  2.18338493e-01]])
      jac: array([-0.00064087,  0.00054932,  0.0005188 , -0.00036621, -0.00012207,
        0.00018311])
  message: 'Optimization terminated successfully.'
     nfev: 203
      nit: 24
     njev: 29
   status: 0
  success: True
        x: array([

In [186]:
a1_mle = np.sqrt(output.hess_inv[0,0])
a2_mle = np.sqrt(output.hess_inv[1,1])
a3_mle = np.sqrt(output.hess_inv[2,2])
bf_mle = np.sqrt(output.hess_inv[3,3])
bp_mle = np.sqrt(output.hess_inv[4,4])
theta_mle = np.sqrt(output.hess_inv[5,5])

In [187]:
print (f"a1_mle: {a1_mle}, a2_mle: {a2_mle}, a3_mle: {a3_mle}, bf_mle: {bf_mle}, bp_mle: {bp_mle}, theta1_mle: {theta_mle}")

a1_mle: 0.0753846329390803, a2_mle: 0.07304691169407931, a3_mle: 0.4587475310236221, bf_mle: 0.086861311649503, bp_mle: 4.3975510881882816, theta1_mle: 0.46726704654099027


In [208]:
rho = np.exp(0.59016408)/(np.exp(0.59016408)+1)

In [209]:
rho

0.6434027923797958

### Case 2: Brands 1 and 2 in one bucket and 3 and 4 in another

In [190]:
def nested_optimization_func_1(x):
    a1, a2, a3, bf, bp, theta1, theta2 = x
    rho1 = np.exp(theta1)/(np.exp(theta1)+1)
    rho2 = np.exp(theta2)/(np.exp(theta2)+1)

    v1 = np.exp((a1 + bf*yogurt_df['Feature 1'] + bp*yogurt_df['Price 1']) / rho1)
    v2 = np.exp((a2 + bf*yogurt_df['Feature 2'] + bp*yogurt_df['Price 2']) / rho1)
    denom1 = v1 + v2

    v3 = np.exp((a3 + bf*yogurt_df['Feature 3'] + bp*yogurt_df['Price 3']) / rho2)
    v4 = np.exp((bf*yogurt_df['Feature 4'] + bp*yogurt_df['Price 4']) / rho2)
    denom2 = v3 + v4
    
    b_1 = np.power(denom1, rho1) / (np.power(denom1, rho1) + np.power(denom2, rho2))
    b_2 = np.power(denom2, rho2) / (np.power(denom1, rho1) + np.power(denom2, rho2))
    
    P1 = (b_1 * v1) / denom1
    P2 = (b_1 * v2) / denom1
    P3 = (b_2 * v3) / denom2
    P4 = (b_2 * v4) / denom2

    loss = (P1 * yogurt_df['Brand 1'] + P2 * yogurt_df['Brand  2'] + P3 * yogurt_df['Brand 3'] + P4 * yogurt_df['Brand 4'])
    log_loss = np.log(loss)
    total_ll = np.sum(log_loss)
    
    return -total_ll

In [191]:
a1, a2, a3, bf, bp, theta1, theta2 = 1, 1, 1, 1, 1, 1, 1
x_init = np.array([a1, a2, a3, bf, bp, theta1, theta2])
output = opt.minimize(nested_optimization_func_1, x_init, options={'gtol': 0.001})

In [192]:
output.x

array([  1.3084072 ,   0.73436123,  -1.92961585,   0.3870842 ,
       -28.19601075,   0.85518528,   0.10108539])

In [193]:
output

      fun: 2654.0983981496765
 hess_inv: array([[ 4.84372359e-03,  1.72424773e-03, -8.14976920e-03,
        -7.86275630e-04, -1.08883955e-01,  3.09611297e-03,
         1.03580820e-02],
       [ 1.72424773e-03,  4.32907735e-03,  7.17088491e-03,
        -1.53340160e-03,  1.19164753e-01, -1.92948701e-02,
        -6.62193317e-03],
       [-8.14976920e-03,  7.17088491e-03,  8.30675148e-02,
        -7.63118465e-03,  7.93415609e-01, -9.11523526e-02,
        -1.18053308e-01],
       [-7.86275630e-04, -1.53340160e-03, -7.63118465e-03,
         8.33915294e-03, -6.24197208e-02,  1.53208475e-02,
         1.21092358e-02],
       [-1.08883955e-01,  1.19164753e-01,  7.93415609e-01,
        -6.24197208e-02,  1.15543900e+01, -1.27090871e+00,
        -8.88274804e-01],
       [ 3.09611297e-03, -1.92948701e-02, -9.11523526e-02,
         1.53208475e-02, -1.27090871e+00,  2.00551112e-01,
         1.07821622e-01],
       [ 1.03580820e-02, -6.62193317e-03, -1.18053308e-01,
         1.21092358e-02, -8.88274804

The parameter estimates from the optimizer are\
a1 = 1.3084072\
a2 = 0.73436123\
a3 = -1.92961585\
bf = 0.3870842\
bp = -28.19601075\
theta1 = 0.85518528\
theta2 = 0.10108539\
Maximized Log Likelihood = -2654.0983981496765

In [194]:
a1_mle = np.sqrt(output.hess_inv[0,0])
a2_mle = np.sqrt(output.hess_inv[1,1])
a3_mle = np.sqrt(output.hess_inv[2,2])
bf_mle = np.sqrt(output.hess_inv[3,3])
bp_mle = np.sqrt(output.hess_inv[4,4])
theta1_mle = np.sqrt(output.hess_inv[5,5])
theta2_mle = np.sqrt(output.hess_inv[6,6])

In [195]:
print (f"a1_mle: {a1_mle}, a2_mle: {a2_mle}, a3_mle: {a3_mle}, bf_mle: {bf_mle}, bp_mle: {bp_mle}, theta1_mle: {theta1_mle}, theta2_mle: {theta2_mle}")

a1_mle: 0.06959686478161163, a2_mle: 0.06579572439957825, a3_mle: 0.2882143555882148, bf_mle: 0.09131896266284897, bp_mle: 3.3991749044180164, theta1_mle: 0.4478293331210372, theta2_mle: 0.4420638025043594


In [210]:
rho1 = np.exp(0.85518528)/(np.exp(0.85518528)+1)
rho2 = np.exp(0.10108539)/(np.exp(0.10108539)+1)

In [211]:
rho1

0.7016537407957196

In [212]:
rho2

0.5252498503743774

### Case 3: Brands 1 and 3 in one bucket and 2 and 4 in another

In [199]:
def nested_optimization_func_2(x):
    a1, a2, a3, bf, bp, theta1, theta2 = x
    rho1 = np.exp(theta1)/(np.exp(theta1)+1)
    rho2 = np.exp(theta2)/(np.exp(theta2)+1)

    v1 = np.exp((a1 + bf*yogurt_df['Feature 1'] + bp*yogurt_df['Price 1']) / rho1)
    v3 = np.exp((a3 + bf*yogurt_df['Feature 3'] + bp*yogurt_df['Price 3']) / rho1)
    
    denom1 = v1 + v3
    
    v2 = np.exp((a2 + bf*yogurt_df['Feature 2'] + bp*yogurt_df['Price 2']) / rho2)
    v4 = np.exp((bf*yogurt_df['Feature 4'] + bp*yogurt_df['Price 4']) / rho2)
    denom2 = v2 + v4
    
    b_1 = np.power(denom1, rho1) / (np.power(denom1, rho1) + np.power(denom2, rho2))
    b_2 = np.power(denom2, rho2) / (np.power(denom1, rho1) + np.power(denom2, rho2))
    
    P1 = (b_1 * v1) / denom1
    P2 = (b_2 * v2) / denom2
    P3 = (b_1 * v3) / denom1
    P4 = (b_2 * v4) / denom2

    loss = (P1 * yogurt_df['Brand 1'] + P2 * yogurt_df['Brand  2'] + P3 * yogurt_df['Brand 3'] + P4 * yogurt_df['Brand 4'])
    log_loss = np.log(loss)
    total_ll = np.sum(log_loss)
    
    return -total_ll

In [200]:
a1, a2, a3, bf, bp, theta1, theta2 = 1, 1, 1, 1, 1, 1, 1
x_init = np.array([a1, a2, a3, bf, bp, theta1, theta2])
output = opt.minimize(nested_optimization_func_2, x_init, options={'gtol': 0.001})

In [201]:
output.x

array([  1.38130384,   0.64233374,  -2.84709944,   0.49156923,
       -36.51054293,   2.21540782,  14.54957194])

In [202]:
output

      fun: 2658.396648614351
 hess_inv: array([[ 4.30904176e-03,  1.85510149e-03, -7.26736744e-03,
        -3.48890387e-05, -5.59672173e-02,  3.83050377e-02,
         7.62898983e-01],
       [ 1.85510149e-03,  2.69116375e-03, -9.51117494e-05,
         1.09696013e-03, -6.48563302e-03,  5.28812929e-03,
         2.03324200e+00],
       [-7.26736744e-03, -9.51117494e-05,  1.47084911e-01,
         1.83869085e-03,  4.91073243e-01, -6.09675795e-01,
        -1.69728884e-01],
       [-3.48890387e-05,  1.09696013e-03,  1.83869085e-03,
         1.13937913e-02,  1.78662378e-02, -1.40323912e-02,
         4.49398458e+00],
       [-5.59672173e-02, -6.48563302e-03,  4.91073243e-01,
         1.78662378e-02,  2.59337062e+00, -2.12468780e+00,
        -1.00914411e+01],
       [ 3.83050377e-02,  5.28812929e-03, -6.09675795e-01,
        -1.40323912e-02, -2.12468780e+00,  2.76910717e+00,
        -6.57411241e+00],
       [ 7.62898983e-01,  2.03324200e+00, -1.69728884e-01,
         4.49398458e+00, -1.00914411e

In [203]:
a1_mle = np.sqrt(output.hess_inv[0,0])
a2_mle = np.sqrt(output.hess_inv[1,1])
a3_mle = np.sqrt(output.hess_inv[2,2])
bf_mle = np.sqrt(output.hess_inv[3,3])
bp_mle = np.sqrt(output.hess_inv[4,4])
theta1_mle = np.sqrt(output.hess_inv[5,5])
theta2_mle = np.sqrt(output.hess_inv[6,6])

In [204]:
print (f"a1_mle: {a1_mle}, a2_mle: {a2_mle}, a3_mle: {a3_mle}, bf_mle: {bf_mle}, bp_mle: {bp_mle}, theta1_mle: {theta1_mle}, theta2_mle: {theta2_mle}")

a1_mle: 0.06564329177505317, a2_mle: 0.05187642773150767, a3_mle: 0.3835165074220923, bf_mle: 0.10674170379170027, bp_mle: 1.610394553674037, theta1_mle: 1.6640634528084732, theta2_mle: 124.50528148876883


In [213]:
rho1 = np.exp(2.21540782)/(np.exp(2.21540782)+1)
rho2 = np.exp(14.54957194)/(np.exp(14.54957194)+1)

In [214]:
rho1

0.901624630655926

In [215]:
rho2

0.9999995200444877

### Case 4: Brands 1 and 4 in one bucket and 2 and 3 in another

In [177]:
def nested_optimization_func_3(x):
    a1, a2, a3, bf, bp, theta1, theta2 = x
    rho1 = np.exp(theta1)/(np.exp(theta1)+1)
    rho2 = np.exp(theta2)/(np.exp(theta2)+1)

    v1 = np.exp((a1 + bf*yogurt_df['Feature 1'] + bp*yogurt_df['Price 1']) / rho1)
    v4 = np.exp((bf*yogurt_df['Feature 4'] + bp*yogurt_df['Price 4']) / rho1)
    
    denom1 = v1 + v4
    
    v2 = np.exp((a2 + bf*yogurt_df['Feature 2'] + bp*yogurt_df['Price 2']) / rho2)
    v3 = np.exp((a3 + bf*yogurt_df['Feature 3'] + bp*yogurt_df['Price 3']) / rho2)

    denom2 = v2 + v3
    
    b_1 = np.power(denom1, rho1) / (np.power(denom1, rho1) + np.power(denom2, rho2))
    b_2 = np.power(denom2, rho2) / (np.power(denom1, rho1) + np.power(denom2, rho2))
    
    P1 = (b_1 * v1) / denom1
    P2 = (b_2 * v2) / denom2
    P3 = (b_2 * v3) / denom2
    P4 = (b_1 * v4) / denom1

    loss = (P1 * yogurt_df['Brand 1'] + P2 * yogurt_df['Brand  2'] + P3 * yogurt_df['Brand 3'] + P4 * yogurt_df['Brand 4'])
    log_loss = np.log(loss)
    total_ll = np.sum(log_loss)
    
    return -total_ll

In [178]:
a1, a2, a3, bf, bp, theta1, theta2 = 1, 1, 1, 1, 1, 1, 1
x_init = np.array([a1, a2, a3, bf, bp, theta1, theta2])
output = opt.minimize(nested_optimization_func_3, x_init, options={'gtol': 0.001})

In [179]:
output.x

array([  1.27044099,   0.59673218,  -1.95510883,   0.44043305,
       -33.75535913,   2.4011839 ,   0.24406051])

In [180]:
output

      fun: 2652.8512628389035
 hess_inv: array([[ 1.49978725e-02,  1.15032233e-02, -1.01743537e-02,
         4.37177623e-04, -3.15287834e-01,  1.55981145e-01,
         1.85289114e-02],
       [ 1.15032233e-02,  1.35288746e-02, -1.72015772e-03,
         1.52345966e-03, -2.18927614e-01,  1.76599462e-01,
         1.29207727e-02],
       [-1.01743537e-02, -1.72015772e-03,  5.33462973e-02,
        -5.33649450e-03,  3.44566764e-01, -6.71096363e-02,
        -6.41702899e-02],
       [ 4.37177623e-04,  1.52345966e-03, -5.33649450e-03,
         1.09605334e-02, -2.10431155e-03,  2.82310930e-02,
         8.34036163e-03],
       [-3.15287834e-01, -2.18927614e-01,  3.44566764e-01,
        -2.10431155e-03,  8.39158811e+00, -3.30539504e+00,
        -4.57520901e-01],
       [ 1.55981145e-01,  1.76599462e-01, -6.71096363e-02,
         2.82310930e-02, -3.30539504e+00,  2.81841801e+00,
         2.17049025e-01],
       [ 1.85289114e-02,  1.29207727e-02, -6.41702899e-02,
         8.34036163e-03, -4.57520901

In [181]:
a1_mle = np.sqrt(output.hess_inv[0,0])
a2_mle = np.sqrt(output.hess_inv[1,1])
a3_mle = np.sqrt(output.hess_inv[2,2])
bf_mle = np.sqrt(output.hess_inv[3,3])
bp_mle = np.sqrt(output.hess_inv[4,4])
theta1_mle = np.sqrt(output.hess_inv[5,5])
theta2_mle = np.sqrt(output.hess_inv[6,6])

In [182]:
print (f"a1_mle: {a1_mle}, a2_mle: {a2_mle}, a3_mle: {a3_mle}, bf_mle: {bf_mle}, bp_mle: {bp_mle}, theta1_mle: {theta1_mle}, theta2_mle: {theta2_mle}")

a1_mle: 0.12246580148711435, a2_mle: 0.11631369069634433, a3_mle: 0.23096817384959414, bf_mle: 0.10469256624501018, bp_mle: 2.8968237974500473, theta1_mle: 1.6788144671087484, theta2_mle: 0.31798520533233976


In [216]:
rho1 = np.exp(2.4011839)/(np.exp(2.4011839)+1)
rho2 = np.exp(0.24406051)/(np.exp(0.24406051)+1)

In [217]:
rho1

0.9169175372601962

In [218]:
rho2

0.5607140541852621

### Case 3 is the only case where the model satisfies IIA

# Logit model with observed heterogeneties

### Including demographics in the model

In [223]:
def demographic_model(x):

    a11, a1i, a1h, a22, a2i, a2h, a33, a3i, a3h, bf, bp, bfi, bfh, bpi, bph = x
    
    a1 = a11 + a1i*yogurt_df["Income"] + a1h*yogurt_df["HH Size"]
    a2 = a22 + a2i*yogurt_df["Income"] + a2h*yogurt_df["HH Size"]
    a3 = a33 + a3i*yogurt_df["Income"] + a3h*yogurt_df["HH Size"]
    
    bfi = bf + bfi*yogurt_df["Income"] + bfh*yogurt_df["HH Size"]
    bpi = bp + bpi*yogurt_df["Income"] + bph*yogurt_df["HH Size"]
    
    v1 = np.exp(a1 + bfi*yogurt_df['Feature 1'] + bpi*yogurt_df['Price 1'])
    v2 = np.exp(a2 + bfi*yogurt_df['Feature 2'] + bpi*yogurt_df['Price 2'])
    v3 = np.exp(a3 + bfi*yogurt_df['Feature 3'] + bpi*yogurt_df['Price 3'])
    v4 = np.exp(bfi*yogurt_df['Feature 4'] + bpi*yogurt_df['Price 4'])
    denom = v1 + v2 + v3 + v4
    
    loss = (v1*yogurt_df['Brand 1'] + v2*yogurt_df['Brand  2'] + v3*yogurt_df['Brand 3'] + v4*yogurt_df['Brand 4']) / denom
    log_loss = np.log(loss)
    total_ll = np.sum(log_loss)
    
    return -total_ll

In [227]:
a11, a1i, a1h, a22, a2i, a2h, a33, a3i, a3h, bf, bp, bfi, bfh, bpi, bph = 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
x_init = np.array([a11, a1i, a1h, a22, a2i, a2h, a33, a3i, a3h, bf, bp, bfi, bfh, bpi, bph])
output = opt.minimize(demographic_model, x_init, options={'gtol': 0.001})

In [228]:
output.x

array([  0.51030182,  -0.09741916,   0.61664267,   1.19037372,
        -0.11850083,   0.19636441,  -1.88498861,  -0.29285122,
         0.37296132,   0.54224479, -40.11435211,  -0.08998898,
         0.21904349,   1.22027358,  -2.45777992])

In [229]:
output

      fun: 2537.4825261512897
 hess_inv: array([[ 3.38326055e-02, -1.53603480e-03, -5.27960572e-03,
         2.09236468e-02, -1.18365211e-03, -2.71291287e-03,
         2.39757443e-02, -1.65413493e-03, -3.28939620e-03,
        -5.64305722e-04, -2.84240797e-02, -1.11126179e-04,
        -4.39302068e-04, -7.32031575e-03,  2.39444907e-02],
       [-1.53603480e-03,  3.09084377e-04, -3.81621958e-04,
        -1.21550868e-03,  1.72101541e-04, -1.54698996e-04,
        -1.88559250e-03,  1.12223584e-04,  2.13312565e-04,
        -8.32521739e-04,  5.23825711e-03, -4.24256019e-05,
         3.61704986e-04, -2.13231988e-03,  1.94448386e-03],
       [-5.27960572e-03, -3.81621958e-04,  3.32297313e-03,
        -2.31302075e-03, -2.37305250e-04,  1.79190636e-03,
        -2.62623012e-03, -2.17498151e-05,  1.15124063e-03,
         2.84331442e-03, -3.83618292e-03,  4.26055834e-05,
        -8.72274690e-04,  3.58950471e-03, -1.46716054e-02],
       [ 2.09236468e-02, -1.21550868e-03, -2.31302075e-03,
         3.0

### Splitting the sample into 4 groups based on median levels of income and household size

In [231]:
yogurt_df[['Income','HH Size']].describe()

,Income,HH Size
count,2430.000000,2430.000000
mean,8.720988,2.802058
std,3.800654,1.173291
min,1.000000,1.000000
25%,6.000000,2.000000
50%,9.000000,3.000000
75%,12.000000,4.000000
max,14.000000,6.000000


In [233]:
grp_1 = yogurt_df[(yogurt_df['Income'] < 9) & (yogurt_df['HH Size'] < 3)]
grp_2 = yogurt_df[(yogurt_df['Income'] >= 9) & (yogurt_df['HH Size'] < 3)]
grp_3 = yogurt_df[(yogurt_df['Income'] < 9) & (yogurt_df['HH Size'] >= 3)]
grp_4 = yogurt_df[(yogurt_df['Income'] >= 9) & (yogurt_df['HH Size'] >= 3)]

In [235]:
def optimization_func_for_grp(x, grp):
    a1, a2, a3, bf, bp = x
    v1 = np.exp(a1 + bf*grp['Feature 1'] + bp*grp['Price 1'])
    v2 = np.exp(a2 + bf*grp['Feature 2'] + bp*grp['Price 2'])
    v3 = np.exp(a3 + bf*grp['Feature 3'] + bp*grp['Price 3'])
    v4 = np.exp(bf*grp['Feature 4'] + bp*grp['Price 4'])
    denom = v1 + v2 + v3 + v4
    loss = (v1*grp['Brand 1'] + v2*grp['Brand  2'] + v3*grp['Brand 3'] + v4*grp['Brand 4'])/denom
    log_loss = np.log(loss)
    total_ll = np.sum(log_loss)
    
    return -total_ll

In [238]:
def BIC(grp):
    x_init = np.array([1, 1, 1, 1, 1])
    results = opt.minimize(optimization_func_for_grp, x_init, grp)
    n = grp.shape[0]
    return np.log(n)*(5)-2*(-results.fun)

In [239]:
BIC(grp_1)

1332.6696512452625

In [240]:
BIC(grp_2)

1251.3502018350791

In [241]:
BIC(grp_3)

990.8763263037957

In [242]:
BIC(grp_4)

1643.0863769764928

BIC varies pretty widely with change in parameters. Especiallt for group 4